In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

from importlib import reload
from matplotlib import rc
from mpl_lego import style
from neurobiases import EMSolver, TriangularModel
from sklearn.decomposition import PCA
%matplotlib inline

In [ ]:
style.use_latex_style()

In [ ]:
K = 1

In [ ]:
# Create triangular model and draw data
tm = TriangularModel(
    model='linear',
    parameter_design='direct_response',
    M=10,
    N=10,
    K=K,
    corr_cluster=0.05,
    corr_back=0.,
    coupling_distribution='gaussian',
    coupling_sparsity=0.5,
    coupling_loc=0,
    coupling_scale=0.25,
    coupling_rng=2332,
    tuning_distribution='gaussian',
    tuning_sparsity=0.5,
    tuning_loc=0,
    tuning_scale=0.25,
    tuning_rng=23456542,
    stim_distribution='uniform')
X, Y, y = tm.generate_samples(n_samples=2000, rng=2332)

In [ ]:
# Run sparse solver
bound = EMSolver(
    X, Y, y, K=K,
    solver='scipy_lbfgs',
    max_iter=1000,
    tol=1e-7,
    penalize_B=False,
    rng=948512,
    fit_intercept=False,
    Psi_transform=None,
    initialization='random')
softplus = bound.copy()
softplus.Psi_transform = 'softplus'
softplus.Psi_tr = softplus.Psi_to_Psi_tr(bound.Psi_tr)
exp = bound.copy()
exp.Psi_transform = 'exp'
exp.Psi_tr = exp.Psi_to_Psi_tr(bound.Psi_tr)

In [ ]:
n_deltas = 20
deltas_init = np.sort(np.insert(np.linspace(-1.1, 1.1, n_deltas), 0, 0))
n_deltas = deltas_init.size
init_params = np.zeros((n_deltas, bound.get_params().size))

for idx, delta in enumerate(deltas_init):
    copy = exp.copy()
    copy.identifiability_transform(delta)
    print(copy.Psi_tr_to_Psi()[0])
    init_params[idx] = copy.get_params(return_Psi=True)

In [ ]:
deltas_fit = np.linspace(-0.18, 0.18, n_deltas)
fit_ident_params = np.zeros((n_deltas, bound.get_params().size))

fit_solver = bound.copy().fit_em()

for idx, delta in enumerate(deltas_fit):
    copy = fit_solver.copy()
    copy.identifiability_transform(delta)
    print(copy.Psi_tr_to_Psi()[0])
    fit_ident_params[idx] = copy.get_params(return_Psi=True)

In [ ]:
fit_params_bound = np.zeros((n_deltas, bound.get_params().size))
fit_params_exp = np.zeros((n_deltas, bound.get_params().size))
fit_params_softplus = np.zeros((n_deltas, bound.get_params().size))

for idx, delta in enumerate(deltas_init):
    print(idx)
    # Bound optimization
    bound_copy = bound.copy()
    bound_copy.identifiability_transform(delta)
    bound_copy.fit_em()
    fit_params_bound[idx] = bound_copy.get_params(return_Psi=True)
    # Exp optimization
    print('exp')
    exp_copy = exp.copy()
    exp_copy.identifiability_transform(delta)
    exp_copy.fit_em()
    fit_params_exp[idx] = exp_copy.get_params(return_Psi=True)
    # Softplus optimization
    print('softplus')
    softplus_copy = softplus.copy()
    softplus_copy.identifiability_transform(delta)
    softplus_copy.fit_em()
    fit_params_softplus[idx] = softplus_copy.get_params(return_Psi=True)

In [ ]:
pca = PCA(2)
fit_transformed = pca.fit_transform(fit_ident_params)
fit_bound_transformed = pca.transform(fit_params_bound)
fit_exp_transformed = pca.transform(fit_params_exp)
fit_softplus_transformed = pca.transform(fit_params_softplus)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(
    fit_transformed[:, 0],
    fit_transformed[:, 1],
    marker='o',
    color='black',
    lw=3,
    alpha=0.2,
    label='Identifiability Family')
ax.plot(
    fit_bound_transformed[:, 0],
    fit_bound_transformed[:, 1],
    marker='o',
    color='C0',
    lw=3,
    alpha=0.75,
    label='No Transform')
ax.plot(
    fit_exp_transformed[:, 0],
    fit_exp_transformed[:, 1],
    marker='o',
    color='C1',
    lw=3,
    alpha=0.75,
    label='Exponential Transform')
ax.plot(
    fit_softplus_transformed[:, 0],
    fit_softplus_transformed[:, 1],
    marker='o',
    color='C2',
    lw=3,
    alpha=0.75,
    label='Softplus Transform')


ax.set_xlabel(r'\textbf{PC 1}', fontsize=18)
ax.set_ylabel(r'\textbf{PC 2}', fontsize=18)
ax.tick_params(labelsize=12)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={'size': 15})

In [ ]:
softplus3 = softplus.copy()
softplus3.solver = 'ow_lbfgs'
softplus3.fit_em(store_parameters=True)

In [ ]:
transformed1 = pca.transform(params_path_real1)
transformed2 = pca.transform(params_path_real2)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(
    transformed1[:, 0],
    transformed1[:, 1],
    marker='o',
    color='red',
    lw=3,
    alpha=0.2,
    label='Initialization 1')
ax.plot(
    transformed2[:, 0],
    transformed2[:, 1],
    marker='o',
    color='gray',
    lw=3,
    alpha=0.2,
    label='Initialization 2')
ax.plot(
    fit_transformed[:, 0],
    fit_transformed[:, 1],
    marker='o',
    color='black',
    lw=3,
    label='Identifiability Family')

ax.set_xlabel(r'\textbf{PC 1}', fontsize=18)
ax.set_ylabel(r'\textbf{PC 2}', fontsize=18)
ax.tick_params(labelsize=12)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={'size': 15})
plt.savefig('optimization_path_identifiability_1.pdf', bbox_inches='tight')

In [ ]:
transformed1 = pca.transform(params_path_real1)
transformed2 = pca.transform(params_path_real2)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(
    transformed1[:, 0],
    transformed1[:, 1],
    marker='o',
    color='red',
    lw=3,
    alpha=0.2,
    label='Initialization 1')
ax.plot(
    transformed2[:, 0],
    transformed2[:, 1],
    marker='o',
    color='gray',
    lw=3,
    alpha=0.2,
    label='Initialization 2')
ax.plot(
    fit_transformed[:, 0],
    fit_transformed[:, 1],
    marker='o',
    color='black',
    lw=3,
    label='Identifiability Family')

ax.set_xlabel(r'\textbf{PC 1}', fontsize=18)
ax.set_ylabel(r'\textbf{PC 2}', fontsize=18)
ax.tick_params(labelsize=12)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), prop={'size': 15})
ax.set_ylim([-0.01, 0.01])
plt.savefig('optimization_path_identifiability_2.pdf', bbox_inches='tight')